Nuestro cliente es Edison. Edison es una reciente compañía de coches autónomos que pretende destronar a Tesla.
Primero importamos la biblioteas necesarioas y definimos archivo de Mongo.

In [1]:
from pymongo import MongoClient
import pandas as pd
import folium

client = MongoClient("mongodb://localhost:27017/")
db = client.db_companies

Ahora filtramos los datos. Lo más importante es que las coordenadas existan y sean diferentes de nulo.

In [2]:

data = pd.DataFrame(list(db.companies.find({'offices.latitude':{'$ne':None},'offices.longitude':{"$exists" : True}},{'category_code':1,'name':1,'number_of_employees':1,'offices':1,'founded_year':1,'products':1 ,'total_money_raised':1,'_id':0})))

En nuestra base de datos vemos qué categorias existen. Como nuestro cliente es una compañía de autos, las categorías más importantes son 'automotive' y 'transportation' (para mover los automóviles). 

In [3]:
data_cate = data[(data['category_code']=='automotive') ]


In [4]:
data_tra = data[data['category_code']=='transportation']

Como los filtros se llevan muchos índices, los reiniciamos.

In [34]:
data_tra=data_tra.reset_index(drop=True)
data_cate=data_cate.reset_index(drop=True)

Definimos unas coordenas para iniciar nuestro mapa.

In [6]:
lat=data_tra.offices[0][0]['latitude']
lon=data_tra.offices[0][0]['longitude']

Y definimos al mapa.

In [37]:
mapa=folium.Map(location=[lat, lon], tiles='openstreetmap', zoom_start=5)

El siguiente paso es hacer una lista de las coordenadas, tanto de los centros automovilísticos, como los de transporte.

In [8]:
coordenadas=[]
coordenadas_t =[]
for i in range(len(data_cate.offices)):
    #print(data_cate.offices[i][0]['latitude'])
    coordenadas.append((data_cate.offices[i][0]['latitude'],data_cate.offices[i][0]['longitude']))
for i in range(len(data_tra.offices)):
    coordenadas_t.append((data_tra.offices[i][0]['latitude'],data_tra.offices[i][0]['longitude']))
    

Y ahora agregamos esos puntos a un mapa.

In [9]:
for e in coordenadas:
       folium.CircleMarker([e[0], e[1]],  icon=folium.Icon(),radio=5,color = 'blue').add_to(mapa)
for e in coordenadas_t:
       folium.CircleMarker([e[0], e[1]], icon=folium.Icon(),radio=5,color = 'red').add_to(mapa)

In [10]:
########################################################################################

In [11]:
#mapa

In [33]:
mapa.save(outfile= "mapa_transportes.html")

Para elegir un buen lugar. Vemos cuál de los puntos de automoviles tiene más transportes cercanos. 

In [12]:
coort = pd.DataFrame(coordenadas_t, columns=['lat','lon'])


In [13]:
coordenadas

[(37.496737, -122.245323),
 (33.133524, -117.254593),
 (42.35888, -71.05682),
 (48.1769, 11.56134),
 (33.526748, -117.177144),
 (34.028701, -118.486966),
 (48.8988968, 2.287772)]

In [14]:
dic_t= []
for i in range(len(coort)):
    dic_t.append({'type': 'Point', 'coordinates': [coort['lat'][i],coort['lon'][i]]})
    

In [16]:
coort['oficina_principal'] = dic_t

In [17]:
first_office = db.first_office.insert_many(coort.to_dict('records'))

In [29]:
db.first_office.create_index([('oficina_principal', '2dsphere')]);

In [19]:
def find_near(geopoint, radio=1000):
    return db.cort.find({
        'oficina_principal':{'$near':{'$geometry':geopoint,
                                     '$maxDistance':radio}}})

In [20]:

radio=10000

In [23]:
lst= []
for i in range(len(coordenadas)):
    try: 
        lst.append([coordenadas[i][0],coordenadas[i][1], find_near({'type':'Point', 'coordinates':[coordenadas[i][0], coordenadas[i][1] ]}, radio).count()])
        #print(type(coordenadas[i][0]),type(coordenadas[i][1]))
    except:
        pass
   

C:\Users\Javier\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


Ahora que ya tenemos las coordenadas y el número de empresas de transportes cercanas, nos quedamos con el lugar con más empresas de transporte cerca.

In [24]:
finales = pd.DataFrame(lst, columns= ['latitud','longitud', 'Transportes cercanos'])

In [25]:
ganador =finales.sort_values('Transportes cercanos').head(1)

Finalmente imprimimos el mapa.

In [30]:
mapa_final=folium.Map(location=[ganador['latitud'], ganador['longitud']], tiles='openstreetmap', zoom_start=5)
folium.Marker([ganador['latitud'], ganador['longitud']],  icon=folium.Icon(),color = 'blue').add_to(mapa_final);

### Y el Lugar ganador es:

In [31]:

mapa_final

In [32]:
mapa_final.save(outfile= "mapa.html")

In [ ]:
###############################